In [22]:
from typing import NamedTuple
import re
from functools import cache

import utils

## Day 4: Scratchcards

[#](https://adventofcode.com/2023/day/4) - We have a pile of scratch cards - we know the numbers we have scratched, and the winning numbers for each card, so we calculate the score for each card.

The score is 1 for the first matching number, and then multiplied by 2 for each extra number.

In [2]:
test: str = """Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11"""

inp = utils.get_input(4, splitlines=False)

We don't need no fancy named tuples, but to keep it easier up ahead, here goes one:

In [3]:
class Game(NamedTuple):
    card: int
    winning: list[int]
    scratched: list[int]
    matches: int

In [4]:
def parse(inp=test, verbose=False):
    games = []
    for line in inp.strip().splitlines():
        a, b = line.split(":")

        left, right = b.split("|")

        num = int(re.findall(r"\d+", a)[0])
        winning_nums = frozenset(int(i) for i in re.findall(r"\d+", left))
        scratched_nums = frozenset(int(i) for i in re.findall(r"\d+", right))
        matches = len(winning_nums & scratched_nums)

        games.append(Game(num, winning_nums, scratched_nums, matches))
    return games


data = parse()
data

[Game(card=1, winning=frozenset({41, 48, 17, 83, 86}), scratched=frozenset({6, 9, 48, 17, 83, 53, 86, 31}), matches=4),
 Game(card=2, winning=frozenset({32, 13, 16, 20, 61}), scratched=frozenset({32, 68, 17, 82, 19, 24, 61, 30}), matches=2),
 Game(card=3, winning=frozenset({1, 44, 53, 21, 59}), scratched=frozenset({1, 69, 72, 14, 16, 82, 21, 63}), matches=2),
 Game(card=4, winning=frozenset({69, 73, 41, 84, 92}), scratched=frozenset({5, 76, 51, 84, 83, 54, 58, 59}), matches=1),
 Game(card=5, winning=frozenset({32, 83, 87, 26, 28}), scratched=frozenset({36, 70, 12, 82, 22, 88, 93, 30}), matches=0),
 Game(card=6, winning=frozenset({72, 13, 18, 56, 31}), scratched=frozenset({35, 67, 36, 74, 10, 11, 77, 23}), matches=0)]

In [5]:
@cache
def score(game: list[int]):
    """returns game score"""
    # wins = len([i for i in game.scratched if i in game.winning])

    if game.matches <= 1:
        return game.matches
    else:
        return 2 ** (game.matches - 1)


score(data[0])

8

In [6]:
def solve(inp=test, verbose: bool = False):
    data = parse(inp)

    scores = [score(game) for game in data]

    return sum(scores)


assert solve(test) == 13  # example answer
solve(inp)

21568

## Part 2

The rules change - each game score gives us a copy of the scratch cards below. E.g card 10 has a scroe of 5, so we get copies of cards 11-15. The copies have the same card number.

This can be done using recursion, but trying a simple way first, where I store a list of the number of copies per game, and go through each game and increase the number of copies.

In [13]:
def solve_2(inp=test, verbose: bool = False):
    data = parse(inp)
    total_games = len(data)

    # we have 1 copy for each game starting out
    copies = [1 for _ in range(0, len(data))]

    for i, game in enumerate(data):
        for _ in range(0, copies[i]):
            # print(list(range(0, copies[i])))
            for i in range(game.card, min(total_games, game.card + game.matches)):
                copies[i] += 1

    if verbose:
        print(sum(copies), copies)
    return sum(copies)


assert solve_2(test) == 30  # example answer
solve_2(inp)

11827296

Above isn't the most ideal solution, as I have some un-necessary loops, but it gets the job done. To get the total score of this game is now fast, as I can use part 1 to get the list of game scores, and multiply it by the number of copies. Numpy makes this easy, or can just some zip inside a list: `[a * b for a, b in zip(scores, copies)]`.

If the cards could also create copies of themselves, the above solution fails.